In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
import pickle

In [2]:
model_yolo = YOLO('YoloV8 Face.pt')

In [3]:
# The name of the person in the video
person_name = "ABDO" 
# The folder name where we will save the face images
output_folder = f"{person_name}_Faces" 

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Folder created: {output_folder}")
else:
    print(f"Folder {output_folder} already exists.")

Folder ABDO_Faces already exists.


In [4]:
# The name of the video file containing the person
video_path = 'ABDO.mp4' 
cap = cv2.VideoCapture(video_path)

count = 0
frame_skip = 5 # It will take a picture every 5 frames for diversity
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    frame_count += 1
    if frame_count % frame_skip != 0:
        continue

    # Detect faces using YOLO
    results = model_yolo(frame, verbose=False)
    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
    
    for box in boxes:
        x1, y1, x2, y2 = box
        # Crop the face from the frame
        face_img = frame[y1:y2, x1:x2]
        
        # Save the face as an image in the folder
        face_filename = os.path.join(output_folder, f"{person_name}_{count}.jpg")
        cv2.imwrite(face_filename, face_img)
        count += 1

cap.release()
print(f"Extracted and saved {count} face images.")

Extracted and saved 355 face images.


In [5]:
X_train = [] # We will store the images here
y_train = [] # We will store the person's name here

# Define the target size for the ArcFace model
TARGET_SIZE = (112, 112) 

# We will loop over all the images we saved in the folder
for img_name in os.listdir(output_folder):
    img_path = os.path.join(output_folder, img_name)
    img = cv2.imread(img_path)
    
    if img is not None:
        # *** FIX: Resize the image to a uniform size ***
        img_resized = cv2.resize(img, TARGET_SIZE)
        
        X_train.append(img_resized)
        y_train.append(person_name)

print(f"Loaded {len(X_train)} images.")

# Convert them to numpy arrays so the model can handle them
# This will work now because all images are (112, 112, 3)
X_train = np.array(X_train)
y_train = np.array(y_train)

print(f"X_train shape: {X_train.shape}")

Loaded 355 images.
X_train shape: (355, 112, 112, 3)


In [6]:

from arcFace import ArcFaceClient 

# Build the model
# This class loads and prepares the model internally
arcface_model = ArcFaceClient()
model = arcface_model.model # This is the model that is ready for use

print("ArcFace model loaded successfully.")
model.summary() # This will print the model summary, and you'll see the output is (None, 512)

ArcFace model loaded successfully.


Model: "ResNet34"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 112, 112,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 114, 114,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      1,728 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_prelu (PReLU) │ (None, 112, 112,  │         64 │ conv1_bn[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 112, 112,  │        256 │ conv1_prelu[0][0] │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_pad  │ (None, 114, 114,  │          0 │ conv2_block1_1_b… │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 112, 112,  │     36,864 │ conv2_block1_1_p… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 112, 112,  │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_pre… │ (None, 112, 112,  │         64 │ conv2_block1_2_b… │
│ (PReLU)             │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_pad  │ (None, 114, 114,  │          0 │ conv2_block1_1_p… │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │      4,096 │ conv1_prelu[0][0] │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_2_p… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_add    │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Add)               │ 64)               │            │ conv2_block1_3_b… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_add

 Total params: 34,165,184 (130.33 MB)

 Trainable params: 34,139,328 (130.23 MB)

 Non-trainable params: 25,856 (101.00 KB)

In [7]:
# *** Second Modification ***
# We will set the matrix size to 512 (for ArcFace) instead of 2622 (for VGG-Face)

embeddings = np.zeros((X_train.shape[0], 512)) 
print(f"Embeddings matrix prepared with shape: {embeddings.shape}")

Embeddings matrix prepared with shape: (355, 512)


In [8]:
for i in range(X_train.shape[0]):
    img = X_train[i]
    
    # Pre-process the image before feeding it to the model
    # (Important: ArcFace expects a (112, 112) BGR image)
    img = cv2.resize(img, (112, 112)) 
    img = img.astype(np.float32)
    img = np.expand_dims(img, axis=0) # To make its shape (1, 112, 112, 3)

    # Calculate the embedding using the new model
    embeddings[i] = model.predict(img, verbose=False)[0, :]
    
    if i % 20 == 0: # Print a message every 20 images
        print(f"Processing image number {i}...")

print("All Embeddings have been calculated.")

Processing image number 0...
Processing image number 20...
Processing image number 40...
Processing image number 60...
Processing image number 80...
Processing image number 100...
Processing image number 120...
Processing image number 140...
Processing image number 160...
Processing image number 180...
Processing image number 200...
Processing image number 220...
Processing image number 240...
Processing image number 260...
Processing image number 280...
Processing image number 300...
Processing image number 320...
Processing image number 340...
All Embeddings have been calculated.


In [9]:
# Convert the embeddings matrix to a DataFrame
embeddings_df = pd.DataFrame(embeddings)

# Add a 'Name' column containing the person's name
embeddings_df['Name'] = y_train

# Save the DataFrame to a CSV file
csv_filename = 'Face Recognition.csv'
embeddings_df.to_csv(csv_filename, index=False)

print(f"Data saved successfully to file {csv_filename}")

Data saved successfully to file Face Recognition.csv
